In [1]:
momFilesPathAndWildCard  = '/mnt/volume/jupyter/szokereso/cron2/szokeresores/momentumraszurt/live_updated3_dict_onlymomentum_data_*_dfUnifiedNanFilteredOnlySomeColsUpdated3.csv'

In [2]:
import datetime
import glob
import pandas as pd

In [3]:
def get_files_sorted_by_date_after_a_date(look_for_this_pattern,cutoffdate):
    
    csvs = glob.glob(look_for_this_pattern)
    datetimes=[datetime.datetime(int(each.split('/')[-1][37:41]),
                                 int(each.split('/')[-1][42:44]),
                                 int(each.split('/')[-1][45:47]),
                                 int(each.split('/')[-1][48:50]),
                                 int(each.split('/')[-1][51:53])) for each in csvs]

    dt_csvs_filtered=[[dt, csv] for dt, csv in zip(datetimes,csvs) if dt >= datetime.datetime(*cutoffdate)]
    
    sorted_filtered_csvs = [csv
                        for _, csv in sorted(
                                         zip([eachpair[0] for eachpair in dt_csvs_filtered],
                                             [eachpair[1] for eachpair in dt_csvs_filtered]))]
    
    return sorted_filtered_csvs

In [4]:
def getFilesOnGivenWeek(dayOfFirstDayOfWeek):
    
    sevenDayAfterDay = dayOfFirstDayOfWeek + datetime.timedelta(days=7)
    
    return list(
    set(get_files_sorted_by_date_after_a_date(momFilesPathAndWildCard,[dayOfFirstDayOfWeek.year,
                                                                       dayOfFirstDayOfWeek.month,
                                                                       dayOfFirstDayOfWeek.day,0]))-\
    set(get_files_sorted_by_date_after_a_date(momFilesPathAndWildCard,[sevenDayAfterDay.year,
                                                                       sevenDayAfterDay.month,
                                                                       sevenDayAfterDay.day,0])))

In [5]:
day=datetime.datetime(2020,7,20,0)
weekfiles = getFilesOnGivenWeek(day)

In [6]:
df = pd.concat(pd.read_csv(weekfile) for weekfile in weekfiles)
df = df[['OUTLET']+[col for col in df.columns if 'momentum' in col]]

In [7]:
s = df.melt('OUTLET').groupby(['value','OUTLET']).size()
s = s.reindex(pd.MultiIndex.from_product(s.index.levels), fill_value=0)

In [8]:
sdf = pd.DataFrame(s)
hits = list(set(sdf.index.get_level_values(level=0)))

In [9]:
sdfs={person:sdf.xs(person, level=0) for person in hits}

In [10]:
def mostfrequent(df):
    outlet  = df.idxmax()
    mentions= df.max()
    return outlet, mentions

In [11]:
res = {person: [mostfrequent(df)[0][0],mostfrequent(df)[1][0]] for person, df in zip(sdfs.keys(), sdfs.values())}

In [12]:
resdf = pd.DataFrame(res).transpose()

In [13]:
resdf.columns = ['OUTLET','TIMES']

In [18]:
resdf.to_csv('MostFeaturedPlatform.csv')

In [19]:
dfMom = df[[col for col in df.columns if 'momentum' in col]]
dfMom = pd.DataFrame(pd.concat(df[col] for col in dfMom.columns))
dfMom = dfMom[0].value_counts()

In [20]:
dfMom.to_csv('mostMentioned.csv')

In [21]:
!pwd

/mnt/volume/jupyter/szokereso/cron2
